# IMDB Text Reviews Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
data = '/Users/lia/Google Drive/UW PhD/Lia Research/PROJECTS/Data/'

In [13]:
weekly_boxoffice = pd.read_csv(data + 'Opus Extract/weekly_boxoffice.csv')
movie_pool = weekly_boxoffice[['movie_odid', 'display_name']].drop_duplicates()
movie_identifiers = pd.read_csv(data +'Opus Extract/movie_identifiers.csv')
movie_imdb_id = movie_identifiers[movie_identifiers['domain']=='IMDB']
movie_imdb_id.rename(columns = {'id':'tconst'}, inplace = True) 
movie_id = pd.merge(movie_pool, movie_imdb_id, how='inner', on=['movie_odid', 'display_name'])
movie_id = movie_id[['tconst']]

# import and merge the imdb public data
title_basics = pd.read_csv(data + 'IMDB/title.basics.tsv',sep='\t')
title_basics = title_basics [title_basics ['titleType'] == 'movie'] # only keep movies
title_ratings = pd.read_csv(data + 'IMDB/title.ratings.tsv',sep='\t') # merge with rating data
title_merge = pd.merge(title_basics, title_ratings, on='tconst') 
title_merge = title_merge[title_merge['startYear']!= '\\N']

movie_space = pd.merge(movie_id, title_merge, how = 'inner', on = 'tconst')
movie_space['startYear'] = movie_space['startYear'].astype(int)
movie_space = movie_space[movie_space['startYear'] >= 1997]

In [38]:
movie_space

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0120338,movie,Titanic,Titanic,0,1997,\N,194,"Drama,Romance",7.8,979418
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",9.0,2130872
3,tt0298148,movie,Shrek 2,Shrek 2,0,2004,\N,93,"Adventure,Animation,Comedy",7.2,391755
5,tt0120915,movie,Star Wars: Episode I - The Phantom Menace,Star Wars: Episode I - The Phantom Menace,0,1999,\N,136,"Action,Adventure,Fantasy",6.5,668058
6,tt0383574,movie,Pirates of the Caribbean: Dead Man's Chest,Pirates of the Caribbean: Dead Man's Chest,0,2006,\N,151,"Action,Adventure,Fantasy",7.3,622828
...,...,...,...,...,...,...,...,...,...,...,...
11135,tt2789014,movie,100 Acres of Hell,100 Acres of Hell,0,2019,\N,85,"Action,Horror",2.9,137
11136,tt9358250,movie,The Disappearance of My Mother,Storia di B. - La scomparsa di mia madre,0,2019,\N,94,Documentary,7.4,90
11137,tt8901942,movie,The Woman Who Loves Giraffes,The Woman Who Loves Giraffes,0,2018,\N,83,Documentary,8.2,82
11138,tt7178226,movie,The Cave,The Cave,0,2019,\N,95,Documentary,8.1,95


In [14]:
movie_space[movie_space['tconst'] == 'tt9844368']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
11105,tt9844368,movie,Out of Liberty,Out of Liberty,0,2019,\N,111,"Drama,Western",8.2,85


In [58]:
#sample_space = movie_space[movie_space['startYear'] == 1997]
#sample_space = movie_space[movie_space['startYear'] == 1998]
#sample_space = movie_space[movie_space['startYear'] == 1999]
#sample_space = movie_space[movie_space['startYear'] == 2000]
#sample_space = movie_space[movie_space['startYear'] == 2001]
#sample_space = movie_space[movie_space['startYear'] == 2002]
#sample_space = movie_space[movie_space['startYear'] == 2003]
#sample_space = movie_space[movie_space['startYear'] == 2004]
#sample_space = movie_space[movie_space['startYear'] == 2005]
#sample_space = movie_space[movie_space['startYear'] == 2006]
#sample_space = movie_space[movie_space['startYear'] == 2007]
#sample_space = movie_space[movie_space['startYear'] == 2008]
#sample_space = movie_space[movie_space['startYear'] == 2009]
#sample_space = movie_space[movie_space['startYear'] == 2010]
#sample_space = movie_space[movie_space['startYear'] == 2011]
#sample_space = movie_space[movie_space['startYear'] == 2012]
#sample_space = movie_space[movie_space['startYear'] == 2013]
#sample_space = movie_space[movie_space['startYear'] == 2014]
#sample_space = movie_space[movie_space['startYear'] == 2015]
#sample_space = movie_space[movie_space['startYear'] == 2016]
#sample_space = movie_space[movie_space['startYear'] == 2017]
#sample_space = movie_space[movie_space['startYear'] == 2018]
sample_space = movie_space[movie_space['startYear'] == 2019]
#sample_space = movie_space[movie_space['tconst'] == 'tt9844368']
len(sample_space)

138

In [59]:
imdbID_space = sample_space['tconst']
length = len(sample_space)
imdbID = []
totalNumReviews = []
spoilerWarning = []
reviewTitles = []
reviewUseful = []
reviewDates = []
userReviews = []
userRates = []
userID = []

i = 0
for x in imdbID_space[0:length]:
    
    print(i, ".", x)
    
    # create the imdbID
    
    url = 'https://www.imdb.com/title/' + x + '/reviews'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Scrape the first page
    # find the total number of the reviews  
    total = "".join(soup.find('div', attrs={'class':'header'}).find('span').text.split()[0].split(","))
    info_base = soup.find_all('div', attrs={'class':'lister-item'})
    selector = 'span svg + span' 
    
    for info in info_base:
        
        imdbID.append(x)
        totalNumReviews.append(total)
        userRates.append(info.select(selector))
        reviewTitles.append(info.find('a', attrs={'class':'title'}).text)
        reviewDates.append(info.find('span', attrs={'class':'review-date'}).text)
        reviewUseful.append(info.find('div', attrs={'class':'actions text-muted'}).text)
        userID.append(info.find('span', attrs={'class':'display-name-link'}).find('a').get('href'))
        userReviews.append(info.find('div', attrs={'class':'text show-more__control'}).text)
        
        if info.find('span', attrs={'class':'spoiler-warning'}) is None:
            
            spoilerWarning.append('None')
        else:          
            spoilerWarning.append(info.find('span', attrs={'class':'spoiler-warning'}).text)
            
        
    # since each page only display 25 items, we need to loop more times
    looptimes = int(np.floor(int(total)/25))
    print('Need to loop', looptimes, 'times')
    
    for j in range(looptimes-1):

        datakey = soup.find('div', attrs={'class':'load-more-data'}).get('data-key')
        url_loadmore = 'https://www.imdb.com/title/' + x + '/reviews/_ajax?paginationKey=' + datakey
        page_loadmore = requests.get(url_loadmore)
        soup_loadmore = BeautifulSoup(page_loadmore.text, 'html.parser')
        
        info_base_loadmore = soup.find_all('div', attrs={'class':'lister-item'})
        selector = 'span svg + span' 
        
        for info_loadmore in info_base_loadmore:
            imdbID.append(x)
            totalNumReviews.append(total)
            userRates.append(info_loadmore.select(selector))
            reviewTitles.append(info_loadmore.find('a', attrs={'class':'title'}).text)
            reviewDates.append(info_loadmore.find('span', attrs={'class':'review-date'}).text)
            reviewUseful.append(info_loadmore.find('div', attrs={'class':'actions text-muted'}).text)
            userID.append(info_loadmore.find('span', attrs={'class':'display-name-link'}).find('a').get('href'))
            userReviews.append(info_loadmore.find('div', attrs={'class':'text show-more__control'}).text)    
            
            if info.find('span', attrs={'class':'spoiler-warning'}) is None:
                spoilerWarning.append('None')
            else:          
                spoilerWarning.append(info_loadmore.find('span', attrs={'class':'spoiler-warning'}))
            
        
        soup = soup_loadmore
        
        print(j)
    
    i = i+1


0 . tt1560220
Need to loop 27 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1 . tt2386490
Need to loop 26 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2 . tt4154664
Need to loop 287 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


32
40 . tt4504044
Need to loop 14 times
0
1
2
3
4
5
6
7
8
9
10
11
12
41 . tt4126476
Need to loop 29 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
42 . tt4648786
Need to loop 11 times
0
1
2
3
4
5
6
7
8
9
43 . tt9024106
Need to loop 24 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
44 . tt7339248
Need to loop 9 times
0
1
2
3
4
5
6
7
45 . tt9356952
Need to loop 1 times
46 . tt7752126
Need to loop 44 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
47 . tt6198946
Need to loop 8 times
0
1
2
3
4
5
6
48 . tt2386668
Need to loop 0 times
49 . tt9019352
Need to loop 0 times
50 . tt6643972
Need to loop 5 times
0
1
2
3
51 . tt7902124
Need to loop 0 times
52 . tt2395469
Need to loop 17 times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
53 . tt8310924
Need to loop 0 times
54 . tt3340446
Need to loop 1 times
55 . tt3009772
Need to loop 1 times
56 . tt8291224
Need to loop 37 times
0

In [60]:
zippedList = zip(imdbID, totalNumReviews, userID, spoilerWarning, reviewTitles, reviewUseful, reviewDates,
                 userReviews, userRates)

imdbReviews = pd.DataFrame(zippedList, columns = ['imdbID', 'totalNumReviews', 'userID', 'spoilerWarning', 
                                                  'reviewTitles', 'reviewUseful', 'reviewDates', 
                                                  'userReviews', 'userRates'])
imdbReviews.to_csv('Output/imdbReviews_2019.csv')